# notebook info

In this notebook you will find a code to parse the data from auto.ru website.
As parsing takes a lot of time i decided to parse only the car brands we have in the test data set in kaggle. 
Test dataset consists about 30K rows. That means we need at least 60K rows in the train dataset to have a 70/30 train/tst split.

The notebook is splitted into two parts:
- 1. create a list of URL' we will parse
- 2. loop which go through all the URL's from the first part and collect the information into one dataset



In [ ]:
#pip install progressbar

In [21]:
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector
import requests
import pandas as pd
from progressbar import ProgressBar
pbar = ProgressBar()

In [ ]:
#define brands
brands_from_kaggle_dataset = ['skoda', 'audi', 'honda', 'volvo', 'bmw', 'nissan', 'infiniti',
       'mercedes', 'toyota', 'lexus', 'volkswagen', 'mitsubishi']

## 1. create list of URL's 

In [ ]:
# create emty list
urls_list_for_parsing_ = []


for brand in pbar(brands_from_kaggle_dataset):
    for i in range(0, 150):     # number of pages for parsing
        parser = 'html.parser'
        resp = requests.get(
            f'https://auto.ru/moskva/cars/{brand}/used/?output_type=list&page={i}')
        http_encoding = resp.encoding if 'charset' in resp.headers.get(
            'content-type', '').lower() else None
        html_encoding = EncodingDetector.find_declared_encoding(
            resp.content, is_html=True)
        encoding = html_encoding or http_encoding
        soup = BeautifulSoup(resp.content, parser, from_encoding=encoding)

        for link in soup.find_all('a', class_='Link ListingItemTitle-module__link', href=True):
            urls_list_for_parsing_.append(link['href'])

## 2. create a dataframe

In [28]:
column_names = ['brand', 'model_name', 'price', 'год выпуска', 'Пробег', 'Кузов', 'Цвет', 'Двигатель', 'Налог',
                'Коробка', 'Привод', 'Руль', 'Состояние', 'Владельцы', 'ПТС', 'Таможня', 'VIN', 'Госномер', 'comment']

df = pd.DataFrame(columns=column_names)

In [ ]:
for url in pbar(urls_part2):
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.encoding = 'utf-8'
        page = BeautifulSoup(response.text, 'html.parser')
        # create empty list with auto characteristics
        auto_characteristics = []
        # add body type
        auto_characteristics.append('brand')
        auto_characteristics.append(url.split('/')[6])
        # add auto name
        auto_characteristics.append('model_name')
        auto_characteristics.append(page.find('h1').text)
        # add price
        auto_characteristics.append('price')
        price = page.find('span', class_='OfferPriceCaption__price').text
        price = price.replace('\xa0', '')
        auto_characteristics.append(price)

        # loop to find al'l other auto characteristics
        for characteristics in page.find_all('span', class_='CardInfoRow__cell'):
            auto_characteristics.append(characteristics.text)

        # add seller comments
        auto_characteristics.append('comment')
        auto_characteristics.append(
            page.find('div', class_='CardDescription__text').text)

        auto_characteristics_dict = dict([(x, y) for x, y in zip(
            auto_characteristics[::2], auto_characteristics[1::2])])
        temp_df = pd.DataFrame([auto_characteristics_dict])
        df = pd.concat([df, temp_df])
    except:
        pass